Authors: Jason Belisario

# Imports
Finish Data cleaning post then watch pandas crash course vid and get on  colab to start with cleaning data sets

1. Access websites and download whatever csv is there & save to GCS bucket

2. Use Trifacta DataPrep to clean

3. Go through DataPrep workflow to look at datasets and whatever cleaning they might need. Save the cleaned dataset in GCS

* Web -> DataFrame to check csv works 
* DataFrame -> CSV 
* CSV -> pushed to GCS (using GCS Client Library)


In [ ]:
# On colab you have to run every session
#!pip install  datapackage   
!pip install fredapi
!pip install datapungi_fed 

In [ ]:
#from datapackage import Package, Resource
import pandas as pd
import numpy as np
from google.cloud import storage 
import os 
from fredapi import Fred
import requests
import datapungi_fed as geoFred #use reg fred api for series & this one for geoFred only 
import datetime

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r''

client = storage.Client()
bucket = client.get_bucket('scout-storage')
print('Done')

Done


# Datahub.io Datasets
**Package -> Resource -> Generator -> List -> DataFrame -> csv in GCS**  

Won't use datahub but keep in case you find any interesting sets on that site 

In [ ]:
# List resources available in datahub package:
package = Package('https://datahub.io/core/finance-vix/datapackage.json')
print(package.resource_names)

In [ ]:
pre_trns_path = 'pre_trns/'

datahub_sets = [{'GDP': 'https://datahub.io/core/gdp-us/datapackage.json'},
                {'CPI': 'https://datahub.io/core/cpi-us/datapackage.json'},
                {'Consumer_World_Inflation': 'https://datahub.io/core/inflation/datapackage.json'},
                {'monthly_10_yr_treasury': 'https://datahub.io/core/bond-yields-us-10y/datapackage.json'},
                {'Case-Shiller': 'https://datahub.io/core/house-prices-us/datapackage.json'},
                {'household_income': 'https://datahub.io/core/household-income-us-historical/datapackage.json'}]

for dataset in datahub_sets:
  for dset_name, dset_link in dataset.items():

    pkg = Package(dset_link)
    resource_save = None
    generator_to_list = []

    for resource in pkg.resources:
        if (resource.descriptor['datahub']['type'] == 'derived/csv' and 
            resource.descriptor['name'] == 'quarter_csv' or 'cpiai_csv' or
            'inflation-consumer_csv' or 'monthly_csv' or 'cities_csv' or
            'household-income-us-historical_csv') :
          #print(resource.read(keyed = True)) 
          resource_save = resource

    # iter() turns Resource object to Generator 
    resource_save = resource_save.iter(keyed = True)  

    for row in resource_save:
      generator_to_list.append(row)

    df = pd.DataFrame(generator_to_list)
    #print(df)

    #When file is saved to bucket with same name, it just overwrites.
    bucket.blob(pre_trns_path + dset_name +'.csv').upload_from_string(df.to_csv(), 'text/csv')

    print('{}.csv uploaded to GCS within {}'.format(dset_name,pre_trns_path))

# Fred.stlouisfed.org Datasets
**FRED -> Series -> DataFrame -> csv in GCS**

> **Monthly**
* Consumer Price Index for All Urban Consumers: All Items in U.S. City Average (CPIAUCSL) - Monthly  
* CBOE 10-Year Treasury Note Volatility Futures (VXTYN) -- frequency = 'm' (original set is daily)  
* 10-Year Treasury Constant Maturity Rate (DGS10) -- frequency = 'm' (original set is daily)  
* 30-Year Treasury Constant Maturity Rate (DGS30) -- frequency = 'm' (original set is daily)  
* 15-Year Fixed Rate Mortgage Average in the United States (MORTGAGE15US) -- frequency = 'm' (original set is weekly)  
* 30-Year Fixed Rate Mortgage Average in the United States (MORTGAGE30US) -- frequency = 'm' (original set is weekly)  
* S&P/Case-Shiller U.S. National Home Price Index (CSUSHPINSA) - Monthly  
* CBOE Volatility Index: VIX (VIXCLS) -- frequency = 'm' (original set is daily)   
* 3-Month London Interbank Offered Rate (LIBOR), based on U.S. Dollar (USD3MTD156N) -- frequency = 'm' (original set is daily)  
* 1-Month London Interbank Offered Rate (LIBOR), based on U.S. Dollar (USD1MTD156N) -- frequency = 'm' (original set is daily)  
* 12-Month London Interbank Offered Rate (LIBOR), based on U.S. Dollar (USD12MD156N) -- frequency = 'm' (original set is daily)  
* University of Michigan: Consumer Sentiment (UMCSENT) - Monthly  
* Housing Starts: Total: New Privately Owned Housing Units Started (HOUST) - Monthly  
* Commercial Bank Interest Rate on Credit Card Plans (TERMCBCCALLNS) - Monthly
* Personal Saving Rate (PSAVERT) - Monthly
* Unemployment Rate (UNRATE) - Monthly
* Continued Claims (Insured Unemployment) (CCSA) -- frequency = 'm' (original set is weekly)
* Total Vehicle Sales (TOTALSA) - Monthly - Seasonally Adjusted Annual Rate
* Advance Retail Sales: Retail (Excluding Food Services) (RSXFS) - Monthly - Seasonally Adjusted

Unemployment by sector. by % and thousands of persons  (Monthly)
https://fred.stlouisfed.org/release/tables?rid=50&eid=4635#snid=4686

> **Quarterly**
* Real Gross Domestic Product (GDPC1) - Quarterly
* Mutual Funds and Exchange-Traded Funds; Corporate Equities; Asset, Flow (BOGZ1FA483064105Q) - Quarterly
* Mutual Funds and Exchange-Traded Funds; Long-Term Debt Securities; Asset, Flow (BOGZ1FA484022605Q) - Quarterly
* New Privately Owned Housing Starts in the United States by Number of Units in Building, 20 or More Units (HOUSTDTA20UMQ) - Quarterly
* Corporate Profits After Tax (without IVA and CCAdj) (CP) - Quarterly

In [ ]:
FRED_API_KEY = ''
fred = Fred(api_key= FRED_API_KEY)

# trns = transformation 
pre_trns_path = 'FRED_pre_trns/'

FRED_monthly = [{'Consumer Price Index for All Urban Consumers': 'CPIAUCSL'}, {'CBOE 10-Year Treasury Note Volatility Futures': 'VXTYN'},
                {'10-Year Treasury Constant Maturity Rate': 'DGS10'}, {'30-Year Treasury Constant Maturity Rate': 'DGS30'},
                {'15-Year Fixed Rate Mortgage Average': 'MORTGAGE15US'}, {'30-Year Fixed Rate Mortgage Average': 'MORTGAGE30US'},
                {'S&P/Case-Shiller U.S. National Home Price Index': 'CSUSHPINSA'}, {'CBOE Volatility Index: VIX': 'VIXCLS'},
                {'3-Month LIBOR, based on U.S. Dollar': 'USD3MTD156N'}, {'1-Month LIBOR, based on U.S. Dollar': 'USD1MTD156N'},
                {'12-Month LIBOR, based on U.S. Dollar': 'USD12MD156N'}, {'University of Michigan: Consumer Sentiment': 'UMCSENT'},
                {'Housing Starts: Total: New Privately Owned Housing': 'HOUST'}, {'Commercial Bank Interest Rate on Credit Card Plans': 'TERMCBCCALLNS'},
                {'Personal Saving Rate': 'PSAVERT'}, {'Unemployment Rate': 'UNRATE'}, {'Continued Claims': 'CCSA'},
                {'Total Vehicle Sales': 'TOTALSA'}, {'Retail Sales': 'RSXFS'}]

# Main monthly, quarterly, and yearly dataframes.  

monthly_join = pd.DataFrame(index = pd.date_range('2010-01-01', datetime.date.today(), freq='MS'))

for dataset in FRED_monthly:
  for dset_name, dset_id in dataset.items():
    monthly_fred_data = fred.get_series(dset_id, observation_start= '2010-01-01', frequency = 'm').rename(dset_name)
  monthly_join = monthly_join.join(monthly_fred_data).interpolate(method = 'time', limit_area = 'inside').fillna(value = 0) 
  
#GCS functions to push df to cloud as csv
# bucket.blob(pre_trns_path + 'Monthly_FRED.csv').upload_from_string(monthly_join.to_csv(), 'text/csv')

# print('Monthly_FRED.csv uploaded to GCS within {}'.format(pre_trns_path))

#pd.DataFrame(monthly_join)

In [ ]:
# FRED Quarterly 
FRED_quarterly = [{'Real Gross Domestic Product': 'GDPC1'},
                  {'Mutual Funds & ETFs; Corporate Equities; Asset, Flow': 'BOGZ1FA483064105Q'},
                  {'Mutual Funds & ETFs; Long-Term Debt Securities; Asset, Flow': 'BOGZ1FA484022605Q'},
                  {'New Privately Owned Housing Starts (20+ units)': 'HOUSTDTA20UMQ'},
                  {'Corporate Profits After Tax ': 'CP'}]

quarterly_join = pd.DataFrame(index = pd.date_range('2010-01-01', datetime.date.today(), freq='MS'))

for dataset in FRED_quarterly:
  for dset_name, dset_id in dataset.items():
    quarterly_fred_data = fred.get_series(dset_id, observation_start= '2010-01-01').rename(dset_name)
  quarterly_join = quarterly_join.join(quarterly_fred_data).interpolate(method = 'time', limit_area = 'inside').fillna(value = 0)

#GCS functions to push df to cloud as csv
# bucket.blob(pre_trns_path + 'Quarterly_FRED.csv').upload_from_string(quarterly_join.to_csv(), 'text/csv')

# print('Quarterly_FRED.csv uploaded to GCS within {}'.format(pre_trns_path))

#pd.DataFrame(quarterly_join).head(n = 20)

#Geo FRED
Only need to have one of the series in the group to get it to search for all counties start_date = 2010-01-01 for now

**Annual**
* msa gdp - Annual - usable ID: NGMP29820

* Estimate of Median Household Income - Annual - usable ID: MHICA06037A052NCEN
* 2018 Bachelor's Degree or Higher (5-year estimate) by County - Annual - usable ID: HC01ESTVC1732017 (only up to 2018, forecast 2019 value by %change from 2017-2018)
* 2018 High School Graduate or Higher (5-year estimate) by County - Annual - usable ID: HC01ESTVC1606071 "" 
* 2018 Per Capita Personal Income by County (dollars) - Annual - usable ID: PCPI06071
* 2018 Personal Income by County (thousands of dollars) - Annual - usable ID: PI06071
* 2019 Resident Population by County (change, thousands of persons) - Annual - usable ID: CASANB1POP
* Combined Violent and Property Crime Incidents Known to Law Enforcement (Known Incidents) - Annual - usable ID: FBITC012086

**Monthly**
* 2020 March Unemployment Rate by County (percent) - Monthly - usable ID: CASANB1URN
* 2020 March Civilian Labor Force by County (persons) - Monthly - usable ID: CASANB1LFN

In [ ]:
geo_pre_trns_path = 'FRED_pre_trns/geo/'
geo_data = geoFred.data(FRED_API_KEY)

geoFRED_county_year = [{'Estimate of Median Household Income ($)': 'MHICA06037A052NCEN'}, {"Bachelor's Degree or Higher (%)": 'HC01ESTVC1732017'},
                       {'High School Graduate or Higher (%)': 'HC01ESTVC1606071'}, {'Per Capita Personal Income ($)': 'PCPI06071'},
                       {'Personal Income (1000 * $)': 'PI06071'},{'Resident Population (thousands of persons)': 'CASANB1POP'},
                       {'Violent and Property Crime Incidents': 'FBITC012086'}]

# geo_year_join = pd.DataFrame(index = pd.date_range('2010-01-01', datetime.date.today(), freq='MS'))

# for dataset in geoFRED_county_year:
#   for dset_name, dset_id in dataset.items():
#     yearly_geoFRED_county = geo_data.geo(series_id = dset_id, start_date = '2010-01-01').drop(columns=['series_id','code']).rename(columns = {'region': dset_name, '_date': 'date'}).dropna()
#     # fix stacked data    
#     yearly_geoFRED_county = yearly_geoFRED_county.pivot(index = 'date', columns = dset_name, values = 'value').add_suffix('_' + dset_name)

#     # change columns from object types to numeric types
#     for col in yearly_geoFRED_county:
#       yearly_geoFRED_county[col] = pd.to_numeric(yearly_geoFRED_county[col], errors='coerce', downcast = 'integer')

#     geo_year_join = geo_year_join.join(yearly_geoFRED_county).interpolate(method = 'time', limit_area = 'inside').fillna(value = 0)
    

#bucket.blob(geo_pre_trns_path + dset_name +'.csv').upload_from_string(yearly_geoFRED_county.to_csv(), 'text/csv')
# print('{}.csv uploaded to GCS within {}'.format(dset_name,geo_pre_trns_path))

#geo_year_join
#pickle to save the bytestream so don't have to run function multiple times to see the same dataframe
# geo_year_pickle = geo_year_join.to_pickle('/content/drive/My Drive/Saf Cap/Scout/Pandas_Pickles/geo_year_join.pickle')
geo_year_pickle2 = pd.read_pickle('/content/drive/My Drive/Saf Cap/Scout/Pandas_Pickles/geo_year_join.pickle')


pd.DataFrame(geo_year_pickle2)

In [ ]:
geo_pre_trns_path = 'FRED_pre_trns/geo/'
geoFRED_county_month = [{'Unemployment Rate (%)': 'CASANB1URN'}, {"Civilian Labor Force (persons)": 'CASANB1LFN'}]

# geo_month_join = pd.DataFrame(index = pd.date_range('2010-01-01', datetime.date.today(), freq='MS'))

# for dataset in geoFRED_county_month:
#   for dset_name, dset_id in dataset.items():
#     monthly_geoFRED_county = geo_data.geo(series_id = dset_id, start_date = '2010-01-01').drop(columns=['series_id','code']).rename(columns = {'region': dset_name, '_date': 'date'}).dropna()
#     # fix stacked data    
#     monthly_geoFRED_county = monthly_geoFRED_county.pivot(index = 'date', columns = dset_name, values = 'value').add_suffix('_' + dset_name)

#     # change columns from object types to numeric types
#     for col in monthly_geoFRED_county:
#       monthly_geoFRED_county[col] = pd.to_numeric(monthly_geoFRED_county[col], errors='coerce', downcast = 'integer')

#     geo_month_join = geo_month_join.join(monthly_geoFRED_county).interpolate(method = 'time', limit_area = 'inside').fillna(value = 0)
    

# bucket.blob(geo_pre_trns_path + dset_name +'.csv').upload_from_string(monthly_geoFRED_county.to_csv(), 'text/csv')
# print('{}.csv uploaded to GCS within {}'.format(dset_name,geo_pre_trns_path))

#geo_month_join

# geo_month_pickle = geo_month_join.to_pickle('/content/drive/My Drive/Saf Cap/Scout/Pandas_Pickles/geo_month_join.pickle')
geo_month_pickle2 = pd.read_pickle('/content/drive/My Drive/Saf Cap/Scout/Pandas_Pickles/geo_month_join.pickle')

# pd.DataFrame(geo_month_pickle2)

In [ ]:
# MSA GDP 
geo_pre_trns_path = 'FRED_pre_trns/geo/'
geoFRED_msa_year = [{'MSA GDP ($)': 'NGMP29820'}]

# msa_annual_join = pd.DataFrame(index = pd.date_range('2010-01-01', datetime.date.today(), freq='MS'))

# for dataset in geoFRED_msa_year:
#   for dset_name, dset_id in dataset.items():
#     yearly_geoFRED_msa = geo_data.geo(series_id = dset_id, start_date = '2010-01-01').drop(columns=['series_id','code']).rename(columns = {'region': dset_name, '_date': 'date'}).dropna()
#     # fix stacked data    
#     yearly_geoFRED_msa = yearly_geoFRED_msa.pivot(index = 'date', columns = dset_name, values = 'value').add_suffix('_' + dset_name)
    
#     # change columns from object types to numeric types
#     for col in yearly_geoFRED_msa:
#       yearly_geoFRED_msa[col] = pd.to_numeric(yearly_geoFRED_msa[col], errors='coerce', downcast = 'integer')

#     msa_annual_join = msa_annual_join.join(yearly_geoFRED_msa).interpolate(method = 'time', limit_area = 'inside').fillna(value = 0)
    

# # bucket.blob(geo_pre_trns_path + dset_name +'.csv').upload_from_string(yearly_geoFRED_msa.to_csv(), 'text/csv')
# # print('{}.csv uploaded to GCS within {}'.format(dset_name,geo_pre_trns_path))

# #msa_annual_join

# msa_annual_pickle = msa_annual_join.to_pickle('/content/drive/My Drive/Saf Cap/Scout/Pandas_Pickles/msa_annual_join.pickle')
msa_annual_pickle2 = pd.read_pickle('/content/drive/My Drive/Saf Cap/Scout/Pandas_Pickles/msa_annual_join.pickle')

# pd.DataFrame(msa_annual_pickle2)

#Joining FRED and GeoFRED data by date index

In [ ]:
pre_trns_path = 'FRED_pre_trns/'
master_join = pd.DataFrame(index = pd.date_range('2010-01-01', datetime.date.today(), freq='MS'))

#df_list = [monthly_join, quarterly_join, geo_year_join, geo_month_join, msa_annual_join]
#df_list = [monthly_join, quarterly_join]

df_list = [geo_year_join]
for df in df_list: 
  master_join = master_join.join(df, how = 'outer')

master_join = master_join

master_join

file_name = 'test_geo.csv'
bucket.blob(pre_trns_path + file_name).upload_from_string(master_join.to_csv(), 'text/csv')
print('{} uploaded to GCS within {}'.format(file_name, pre_trns_path))

test_geo.csv uploaded to GCS within FRED_pre_trns/


# Other economic data sources

Mainly Federal Reserve Economic Indicators
  1. Consumer Credit 
  2. Industrial Production and Capacity Utilization
  3. Real Estate Owned (REO) 

Housing Credit Availability Index (HCAI) - https://www.urban.org/sites/default/files/hcai2019q4.xlsx  

The HCAI measures the percentage of owner-occupied home purchase loans that are likely to default—that is, go unpaid for more than 90 days past their due date.  

A lower HCAI indicates that lenders are unwilling to 
tolerate defaults and are imposing tighter lending standards, making it harder to get a loan.  

A higher HCAI indicates that lenders are willing to tolerate defaults and are taking more risks, making it easier to get a loan.

* Consumer Credit Index (from OECD) - have to delete any sources that aren't USA location. Might just not use b/c already have UMich one 
* Real Estate Owned (REO) - i.e. Total real estate loans owned and securitized by finance companies
* Consumer Credit Outstanding (CCO)
* Industrial Production: Market and Industry Groups (IP)
* Capacity Utilization (CU)

**Link -> Dataframe -> GCS as csv**

In [ ]:
econ_pre_trns_path = 'econ_pre_trns/'

# read_excel
HCAI_url = [{'Housing Credit Availability Index': 'https://www.urban.org/sites/default/files/hcai2019q4.xlsx'}] 

HCAI_join = pd.DataFrame(index = pd.date_range('2010-01-01', datetime.date.today(), freq='MS'))
def create_quarters(row):
   if row['Quarter'] == 1 :
      return row['Year'] + '-01-01'
   elif row['Quarter'] == 2 :
      return row['Year'] + '-04-01'
   elif row['Quarter'] == 3 :
      return row['Year'] + '-07-01'
   elif row['Quarter'] == 4 :
      return row['Year'] + '-10-01'

for dataset in HCAI_url:
  for dset_name, dset_url in dataset.items():
    
    haci_df = pd.read_excel(dset_url)
    haci_df.columns = haci_df.iloc[2] #set column names to a specific row 
    haci_df = haci_df.drop(index = [0,1,2], columns= 'highlight')

    # Create dates out of Year and Quarter num original columns 
    haci_df['Year'] = haci_df['Year'].apply(str)
    haci_df['Date'] = haci_df.apply(lambda row: create_quarters(row), axis=1)
    haci_df['Date'] = pd.to_datetime(haci_df['Date'])

    haci_df = haci_df.drop(columns= ['Year', 'Quarter']).set_index('Date').rename(columns = {'Total\nRisk': 'Total Risk', 'Borrower\nRisk': 'Borrower Risk', 'Product\nRisk': 'Product Risk'})
    HCAI_join = HCAI_join.join(haci_df, how = 'inner') # later maybe join outer to monthly df which allows for easier join to other monthly datasets if necessary 

HCAI_join
# bucket.blob(econ_pre_trns_path + dset_name +'.csv').upload_from_string(HCAI_join.to_csv(), 'text/csv')
# print('{}.csv uploaded to GCS within {}'.format(dset_name,econ_pre_trns_path))

In [ ]:
# read_csv
econ_pre_trns_path = 'econ_pre_trns/'
fed_file = 'joined_fed.csv'
Fed_urls = [{'REO_FED': 'https://www.federalreserve.gov/datadownload/Output.aspx?rel=G20&series=382ce44f124afc993f2b566c6c2cd4ef&lastobs=&from=&to=&filetype=csv&label=include&layout=seriescolumn&type=package'},
            {'Consumer Credit Outstanding_FED':'https://www.federalreserve.gov/datadownload/Output.aspx?rel=G19&series=47b3133fcba3957706678b2a55cb5a97&lastobs=&from=&to=&filetype=csv&label=include&layout=seriescolumn&type=package'},
            {'Industrial Production_FED':'https://www.federalreserve.gov/datadownload/Output.aspx?rel=G17&series=5d88c03b0036f0334d78f6bafefc5101&lastobs=&from=&to=&filetype=csv&label=include&layout=seriesrow&type=package'},
            {'Capacity Utilization_FED':'https://www.federalreserve.gov/datadownload/Output.aspx?rel=G17&series=f213ee970e317c0f3cc80e775d35b8e5&lastobs=&from=&to=&filetype=csv&label=include&layout=seriesrow&type=package'}]

#Fed_urls = [{'REO_FED': 'https://www.federalreserve.gov/datadownload/Output.aspx?rel=G20&series=382ce44f124afc993f2b566c6c2cd4ef&lastobs=&from=&to=&filetype=csv&label=include&layout=seriescolumn&type=package'}]

fed_join = pd.DataFrame(index = pd.date_range('2010-01-01', datetime.date.today(), freq='MS'))
for dataset in Fed_urls:
  for dset_name, dset_url in dataset.items():
    if dset_name == 'Capacity Utilization_FED' or dset_name == 'Industrial Production_FED':
      fed_df = pd.read_csv(dset_url).drop(columns = ['Unit:', 'Multiplier:', 'Currency:', 'Unique Identifier:', 'Series Name:']).rename(columns = {'Descriptions:': 'Time'}).set_index('Time').T
      fed_df.index = fed_df.index + '-01'
      fed_df.index = pd.to_datetime(fed_df.index)
    else: 
      fed_df = pd.read_csv(dset_url).drop(index = [0,1,2,3,4]).rename(columns = {'Series Description': 'Time'}).set_index('Time') 
      fed_df.index = fed_df.index + '-01'
      fed_df.index = pd.to_datetime(fed_df.index)
    
    fed_join = fed_join.join(fed_df, how = 'inner').replace('ND', np.nan).apply(pd.to_numeric)  

fed_join
#bucket.blob(econ_pre_trns_path + fed_file).upload_from_string(fed_join.to_csv(), 'text/csv')
#print('{} uploaded to GCS within {}'.format(fed_file,econ_pre_trns_path))

In [ ]:
CCI_url = [{'CCI': 'https://stats.oecd.org/sdmx-json/data/DP_LIVE/.CCI.../OECD?contentType=csv&detail=code&separator=comma&csv-lang=en'}]

for dataset in CCI_url:
  for dset_name, dset_url in dataset.items():
    cci_df = pd.read_csv(dset_url)

cci_df

# bucket.blob(econ_pre_trns_path + dset_name +'.csv').upload_from_string(cci_df.to_csv(), 'text/csv')
# print('{}.csv uploaded to GCS within {}'.format(dset_name,econ_pre_trns_path))

In [ ]:
# send request 
econ_pre_trns_path = 'econ_pre_trns/'
pensions_file = 'pensions.csv'
pensions_url = [{'Public Pensions': 'http://publicplansdata.org/api/?q=QVariables&variables=PlanName,fy,EQTotal_Actl,PETotal_Actl,HFTotal_Actl,COMDTotal_Actl,RETotal_Actl,AltMiscTotal_Actl,CashTotal_Actl,OtherTotal_Actl,EQTotal_Trgt,FITotal_Trgt,PETotal_Trgt,HFTotal_Trgt,COMDTotal_Trgt,RETotal_Trgt,AltMiscTotal_Trgt,CashTotal_Trgt,OtherTotal_Trgt&filterfystart=2001&filterfyend=2020&format=json'}]

for dataset in pensions_url:
  for dset_name, dset_url in dataset.items():
    r = requests.get(url = dset_url)
    data = r.json()
    pensions_df = pd.DataFrame(data)
    pensions_df = pensions_df.drop(columns = ['status', 'date', 'q', 'params', 'recordcount']).rename(columns = {'fy': 'Time'}).set_index('Time').dropna(how = 'all')
    
    pensions_df.index = pensions_df.index + '-01'
    pensions_df.index = pd.to_datetime(pensions_df.index)

    pensions_df = pensions_df.fillna(0)
    #.add_suffix('_' + dset_name)
    pensions_df = pensions_df.pivot(columns = 'PlanName',)

#pensions_df
bucket.blob(econ_pre_trns_path + pensions_file).upload_from_string(pensions_df.to_csv(), 'text/csv')
print('{} uploaded to GCS within {}'.format(pensions_file,econ_pre_trns_path))

pensions.csv uploaded to GCS within econ_pre_trns/


# Zillow Real Estate Data

 

**Zillow Home Value Index (ZHVI)** 
* ZHVI All Homes (SFR, Condo/Co-op) Time-Series ($) - City - http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv   

* ZHVI All Homes (SFR, Condo/Co-op) Time-Series ($) - County - http://files.zillowstatic.com/research/public/County/County_Zhvi_AllHomes.csv

* ZHVI All Homes (SFR, Condo/Co-op) Time-Series ($) - MSA - http://files.zillowstatic.com/research/public/Metro/Metro_Zhvi_AllHomes.csv

* ZHVI Single-Family Homes Time Series - City - http://files.zillowstatic.com/research/public/City/City_Zhvi_SingleFamilyResidence.csv

* ZHVI Single-Family Homes Time Series - County - http://files.zillowstatic.com/research/public/County/County_Zhvi_SingleFamilyResidence.csv

* ZHVI Single-Family Homes Time Series - MSA - http://files.zillowstatic.com/research/public/Metro/Metro_Zhvi_SingleFamilyResidence.csv

**Zillow Rent Index (ZRI)**
* ZRI Time-Series: Multifamily, SFR, Condo/Co-op ($) - City - http://files.zillowstatic.com/research/public/City/City_Zri_AllHomesPlusMultifamily.csv

* ZRI Time-Series: Multifamily, SFR, Condo/Co-op ($) - County - http://files.zillowstatic.com/research/public/County/County_Zri_AllHomesPlusMultifamily.csv

* ZRI Time-Series: Multifamily, SFR, Condo/Co-op ($) - MSA - http://files.zillowstatic.com/research/public/Metro/Metro_Zri_AllHomesPlusMultifamily.csv

* ZRI Time-Series: Multifamily ($) - City - http://files.zillowstatic.com/research/public/City/City_Zri_MultiFamilyResidenceRental.csv

* ZRI Time-Series: Multifamily ($) - County - http://files.zillowstatic.com/research/public/County/County_Zri_MultiFamilyResidenceRental.csv

* ZRI Time-Series: Multifamily ($) - MSA - http://files.zillowstatic.com/research/public/Metro/Metro_Zri_MultiFamilyResidenceRental.csv

**Misc.**
* Age of Inventory (Days) - MSA - http://files.zillowstatic.com/research/public/Metro/MedianAgeOfInventory_NSA_AllHomes_Metro.csv

In [ ]:

zillow_pre_trns_path = 'Zillow_pre_trns/'

zillow_sfh_condo_values = [{'SFR, Condo Values - City':'http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv'},
                           {'SFR, Condo Values - County':'http://files.zillowstatic.com/research/public/County/County_Zhvi_AllHomes.csv'},
                           {'SFR, Condo Values - MSA':'http://files.zillowstatic.com/research/public/Metro/Metro_Zhvi_AllHomes.csv'}]

for dataset in zillow_sfh_condo_values:
  for dset_name, dset_url in dataset.items():
    zillow_sfh_condo_values_df = pd.read_csv(dset_url)

    bucket.blob(zillow_pre_trns_path + dset_name +'.csv').upload_from_string(zillow_sfh_condo_values_df.to_csv(), 'text/csv')

    print('{}.csv uploaded to GCS within {}'.format(dset_name,zillow_pre_trns_path))

zillow_sfh_values = [{'Single-Family Homes Values - City': 'http://files.zillowstatic.com/research/public/City/City_Zhvi_SingleFamilyResidence.csv'},
                     {'Single-Family Homes Values - County': 'http://files.zillowstatic.com/research/public/County/County_Zhvi_SingleFamilyResidence.csv'},
                     {'Single-Family Homes Values - MSA': 'http://files.zillowstatic.com/research/public/Metro/Metro_Zhvi_SingleFamilyResidence.csv'}]

for dataset in zillow_sfh_values:
  for dset_name, dset_url in dataset.items():
    zillow_sfh_values_df = pd.read_csv(dset_url)

    bucket.blob(zillow_pre_trns_path + dset_name +'.csv').upload_from_string(zillow_sfh_values_df.to_csv(), 'text/csv')

    print('{}.csv uploaded to GCS within {}'.format(dset_name,zillow_pre_trns_path))

zillow_Multi_Condo_rent = [{'Multifamily, SFR, Condo Rent - City': 'http://files.zillowstatic.com/research/public/City/City_Zri_AllHomesPlusMultifamily.csv'},
                           {'Multifamily, SFR, Condo Rent - County': 'http://files.zillowstatic.com/research/public/County/County_Zri_AllHomesPlusMultifamily.csv'},
                           {'Multifamily, SFR, Condo Rent - MSA': 'http://files.zillowstatic.com/research/public/Metro/Metro_Zri_AllHomesPlusMultifamily.csv'}]   

for dataset in zillow_Multi_Condo_rent:
  for dset_name, dset_url in dataset.items():
    zillow_Multi_Condo_rent_df = pd.read_csv(dset_url, encoding= "ISO-8859-1")

    bucket.blob(zillow_pre_trns_path + dset_name +'.csv').upload_from_string(zillow_Multi_Condo_rent_df.to_csv(), 'text/csv')

    print('{}.csv uploaded to GCS within {}'.format(dset_name,zillow_pre_trns_path))


zillow_Multi_rent = [{'Multifamily Rent - City': 'http://files.zillowstatic.com/research/public/City/City_Zri_MultiFamilyResidenceRental.csv'},
                     {'Multifamily Rent - County': 'http://files.zillowstatic.com/research/public/County/County_Zri_MultiFamilyResidenceRental.csv'},
                     {'Multifamily Rent - MSA': 'http://files.zillowstatic.com/research/public/Metro/Metro_Zri_MultiFamilyResidenceRental.csv'}]  

for dataset in zillow_Multi_rent:
  for dset_name, dset_url in dataset.items():
    zillow_Multi_rent_df = pd.read_csv(dset_url)

    bucket.blob(zillow_pre_trns_path + dset_name +'.csv').upload_from_string(zillow_Multi_rent_df.to_csv(), 'text/csv')

    print('{}.csv uploaded to GCS within {}'.format(dset_name,zillow_pre_trns_path))


zillow_inventory_age = [{'Age of Inventory (Days) - MSA': 'http://files.zillowstatic.com/research/public/Metro/MedianAgeOfInventory_NSA_AllHomes_Metro.csv'}]

for dataset in zillow_inventory_age:
  for dset_name, dset_url in dataset.items():
    zillow_inventory_age_df = pd.read_csv(dset_url)

    bucket.blob(zillow_pre_trns_path + dset_name +'.csv').upload_from_string(zillow_inventory_age_df.to_csv(), 'text/csv')

    print('{}.csv uploaded to GCS within {}'.format(dset_name,zillow_pre_trns_path))


# Test block

In [ ]:
capacity_util = 'https://www.federalreserve.gov/datadownload/Output.aspx?rel=G17&series=f213ee970e317c0f3cc80e775d35b8e5&lastobs=&from=&to=&filetype=csv&label=include&layout=seriesrow&type=package'

df = pd.read_csv(capacity_util)
#drop columns and transpose so date is index 
df = df.drop(columns= ['Unit:', 'Multiplier:', 'Currency:', 'Unique Identifier:', 'Series Name:']).set_index(['Descriptions:']).T

#only use rows where timestamp index >= Jan 1, 2010
master_join.loc[(master_join.index >= pd.Timestamp('2010-01-01'))]


# match CBG's of a market to that market. Get the 130 ish markets that we will cover and match MSA's, CBG's, Counties, to our Yardi Markets

Split up demographic data to the Yardi Markets. 

5/30 - all csv's merged from census data. Deleted any columns with margin of error data. Kept only estimate columns.  
NaN's in the first batch of columns. Figure out later why this is. - *fixed with right join*   
After merge. Index has .0 in it. 
otherwise all cols & rows seem to be there




FIPS code: 12 digits   
1-2 digits = state  
3-5 digits = county   
6-12 digits don't matter (tract & block group)  


I want to get the Census block group data and group it under its correct yardi market 
yardi market -> county name -> county ID 

Loop through list of yardi market county ID's (must be same as Census ID's), for every ID, create new dataframe that is comprised of all the CBG rows in county_id col with said yardi_county_id using Groupby.

Write each of those dataframes to csv a folder for yardi_market_census_data in GCS. Should be around 130 csv's for all of the yardi markets. 


Census Block group -> County ID 

Should i get our list of markets and write an excel file with their corresponding counties? 
Then delete counties that don't match up to any of the counties that correspond to our markets? 
 
Dropped CBG file 22. Issue from source with Index not being correct and didn't want to copy index from other CBG file and assume those indices would be correct. Also the actual data didn't seem useful so doesn't seem like a real loss. (number of disabled persons on food stamps if i'm not mistaken) 


In [ ]:
!pip install gcsfs
!pip install partd

In [ ]:
from io import BytesIO
from google.cloud import storage
import pandas as pd
import os 
import dask.dataframe as dd
import gcsfs
import partd
from itertools import repeat
import json
from functools import reduce

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r''

storage_client = storage.Client()
project_id = ''
token_file = ''
fs = gcsfs.GCSFileSystem(project = project_id)
gcs_bucket_name = 'scout-storage'
file_name = '/census/cbg_b00.csv' # get first csv to create index 

print('Done')

Done


In [ ]:
# dataframe's that will be concatenated and merged written here & saved to pickles 
census_index = dd.read_csv('gs://'+ gcs_bucket_name + file_name, storage_options={'token': token_file, 'project': project_id})
census_index = census_index.drop(columns = ['B00001e1', 'B00001m1', 'B00002e1', 'B00002m1'])
census_index['census_block_group'] = census_index['census_block_group'].astype(int).astype(str)

# parse state and county id's 
state_county_id = census_index.compute() 
state_county_id['state_id'] = state_county_id.apply(lambda row: row['census_block_group'][0:2] if len(row['census_block_group']) == 12 else row['census_block_group'][0], axis=1)
state_county_id['county_id'] = state_county_id.apply(lambda row: row['census_block_group'][2:5] if len(row['census_block_group']) == 12 else row['census_block_group'][1:4], axis=1)
state_county_id = state_county_id.set_index('census_block_group')

census_index = census_index.set_index('census_block_group')
census_index = census_index.compute() 

state_county_id.to_pickle('/content/drive/My Drive/Saf Cap/Scout/Pandas_Pickles/state_county_id.pickle')
census_index.to_pickle('/content/drive/My Drive/Saf Cap/Scout/Pandas_Pickles/census_index.pickle')

In [ ]:
def get_blob_list(**kwargs):
  blobs = storage_client.list_blobs('scout-storage', prefix = 'census/cbg')
  num_blobs = kwargs.get('num_blobs', 21)
  i = 0
  blob_list = []
  for blob in blobs:  
    if i < num_blobs: 
      blob_list.append(blob.name)
    else: 
      break
    i += 1
    
  return blob_list

#return cbg files as pandas dataframes
def cbg_files(blob_name):
  project_id = 'sigma-cairn-275317'
  token_file = '/content/drive/My Drive/0 - GCP Credentials/Scout-19edc12a6cf5.json'
  gcs_bucket_name = 'scout-storage'

  #print(blob_name)
  df = dd.read_csv('gs://'+ gcs_bucket_name + '/' + blob_name, storage_options={'token': token_file, 'project': project_id}, assume_missing = True)
  droplist = [col_name for col_name in df.columns if col_name[-2] == 'm' or col_name[-3] == 'm'] 
  df = df.drop(droplist, axis = 1)
  df['census_block_group'] = df['census_block_group'].astype(int).astype(str)
  df = df.set_index('census_block_group') # set index to be able to merge 
  return df.compute()

In [ ]:
#blobs with state & county id's appended and saved as pickles 
state_county_id_df = pd.read_pickle('/content/drive/My Drive/Saf Cap/Scout/Pandas_Pickles/state_county_id.pickle')

census_blobs = get_blob_list(num_blobs = 21) # get census blob list
print(census_blobs)

# for blob_name in census_blobs:
#   census_cbg_df = cbg_files(blob_name)
#   census_cbg_df = pd.concat([census_cbg_df, state_county_id_df], axis = 1) # concat with state/county id's 
  
#   # [7:-4] to get rid of the census/ & .csv file designation 
#   census_cbg_df.to_pickle('/content/drive/My Drive/Saf Cap/Scout/Pandas_Pickles/cbg_blob_list_ids/' + blob_name[7:-4] + '.pickle')
#   print(blob_name[7:-4] + ' uploaded')

In [ ]:
cbg_list = []
for blob_name in census_blobs:
 
   cbg_list.append(pd.read_pickle('/content/drive/My Drive/Saf Cap/Scout/Pandas_Pickles/cbg_blob_list_ids/' + blob_name[7:-4] + '.pickle'))

print('Done')
#print(cbg_list)

In [ ]:
client = storage.Client()
bucket = client.get_bucket('scout-storage')

# Opening markets_dict JSON file 
with open("") as json_file: 
    markets = json.load(json_file) 

x = 0
county_df_list = []
market_df_list = []

def group_by_market(census_df_list, markets_dict):
  for market in markets_dict:
    # logic for cbg file work here. just run fxn one line return census_cbg_df
    for cbg_df in census_df_list: # not the issue
      for market_name, market_details in market.items():
        try:
          for x in range(4):

            x += 1
            x = str(x)
            state_num = 'state_' + x # to loop through keys state_1 through state_4 if they exist 
            state_id = market_details[state_num]['state_id']
            counter = 1

            for county_name, county_id in market_details[state_num]['counties'].items():
              county_id = county_id.zfill(3) # pad county id's with 0's if county_id < 3 characters. 
              concat_df = cbg_df.loc[(cbg_df['state_id'] == state_id) & (cbg_df['county_id'] == county_id)]
              concat_df = concat_df.drop(columns= ['state_id','county_id'])
              county_df_list.append(concat_df)

        except:
          pass
        
        for df in county_df_list:
          df = df.reset_index(inplace=True)
          
        #returns concatenated df FOR THAT MARKET using 1 cbg file. should break out of loop and go to next item on CBG_list list
        concat_result_df = pd.concat(county_df_list, axis = 1, join = 'inner') #problem in concat lines 
        concat_result_df = concat_result_df.loc[:,~concat_result_df.columns.duplicated()]
        county_df_list.clear() # clear items in list as to not conc. all counties of each market
        #for each market & each CBG file, Append that CBG parsed file to list 
      market_df_list.append(concat_result_df) 
  
    market_result_df = pd.concat(market_df_list, axis = 1, join = 'inner')#verify_integrity = True
    market_result_df = market_result_df.loc[:,~market_result_df.columns.duplicated()].rename(columns= {'index': 'census_block_group'})
    market_df_list.clear()

    print(market_result_df)
    bucket.blob('yardi_market_census/' + market_name + '.csv').upload_from_string(market_result_df.to_csv(), 'text/csv')
    print(market_name + '.csv uploaded')
    

group_by_market(cbg_list, markets)

In [ ]:
# checking if all 220k rows from each cbg file are the same 
from itertools import cycle
project_id = ''
token_file = ''
fs = gcsfs.GCSFileSystem(project=project_id)
gcs_bucket_name = 'scout-storage'

cycle_blobs = cycle(census_blobs)
next_file = next(cycle_blobs) #start the iterating at file 00 

for blob_name in census_blobs:
  
  #[7:-4] to get rid of the census/ & .csv file designation 
  
  print(blob_name)
  next_file = next(cycle_blobs)
  df = pd.read_csv('gs://'+ gcs_bucket_name +'/' + blob_name)
  df['census_block_group'] = df['census_block_group'].astype(int).astype(str) 
  df = df.loc[:, df.columns.intersection(['census_block_group'])]

  next_df = pd.read_csv('gs://'+ gcs_bucket_name +'/' + next_file)# 01
  print('next file: ' + next_file)
  next_df['census_block_group'] = next_df['census_block_group'].astype(int).astype(str) 
  next_df = next_df.loc[:, next_df.columns.intersection(['census_block_group'])]

  test = df.equals(next_df)
  
  print(test)

# REIS

Clean and join REIS data

For markets without a cap rate, add a column for cap rate and fill it with NaN's or 0's.


Just do the regular fix and join to REIS and then later once you have all of them joined, iterate through all lists to get an average of the cap rate for each month. then use that to populate list of the ones with 0's in the rates. Do same with expense %. 

In [1]:
import pandas as pd
import os
import datetime
import re
import matplotlib.pyplot as plt

In [ ]:
#pd.set_option('display.max_rows', 300)
REIS_test_path = ''
trends_path = ''

def transform_trends(path):
  try: 
    trends_df = pd.read_csv(path)
  except:
    trends_df = pd.read_excel(path)

  if 'Cap Rate' in trends_df:
    try:
      trends_df['Cap Rate'] = trends_df['Cap Rate'].replace({'–': 0})
    except:
      pass

  trends_df['Year'] = pd.to_datetime(trends_df['Year'], format='%Y')
  trends_df = trends_df.set_index('Year')
  upsampled = trends_df.resample('MS').interpolate()
  return upsampled

transform_trends(trends_path)#.tail(n = 20)

In [5]:
#REIS Report 
pd.set_option('display.max_rows', 300)

REIS_test_path = ''

def transform_REIS(path):
  try: 
    REIS_df = pd.read_csv(path)
  except:
    REIS_df = pd.read_excel(path)

  REIS_df = REIS_df.loc[(REIS_df['Building Class'] == 'All')]
  REIS_df = REIS_df[REIS_df['Period'].str.contains('Q')]
  REIS_df['Period'] = REIS_df['Period'].replace({'Q1': '0101', 'Q2': '0401', 'Q3': '0701', 'Q4': '1001'})
  REIS_df['Date'] = REIS_df['Period'] + REIS_df['Year'].astype(str)
  REIS_df['Date'] = pd.to_datetime(REIS_df['Date'],format='%m%d%Y', dayfirst = True)
  REIS_df = REIS_df.drop(columns= ['Sector', 'Metro Code', 'Building Class', 'Year', 'Period', 'State' ]).set_index('Date').resample('MS').interpolate()
  REIS_df['Market Name'] = REIS_df['Market Name'].fillna(method = 'pad')
  return REIS_df

#transform_REIS(REIS_test_path)

In [7]:
#RE parse for market name b/c only will use sum function if there are more than 1 files in a directory with REIS or trends name. 
#RE parse market name and make file name marketname_REIS_joined.csv 
#call sum function after both transform and join functions are called 



def sum_df_list(df_list):
  return pd.concat(df_list, axis = 1).groupby(level = 0, axis = 1).sum()

#sum_df_list(list_of_dfs)

In [8]:

#full_join.to_csv(path_or_buf='/content/drive/My Drive/test.csv') 

def join_REIS_trends (REIS_df, trends_df):
  REIS_trends_df = REIS_df.join(trends_df, how = 'left')
  return REIS_trends_df

In [ ]:
#expense % and cap rate for all markets averaged out and joined
test_dir = ''

exp_and_cap_list = [] # list of all df's with caprate & expense 
for dirpath, dirnames, filenames in os.walk(test_dir):
  trends_path_list = []
  REIS_path_list = []
  
  trends_df_list = []
  REIS_df_list = []

  
  for file in filenames:
    
    REIS_path = dirpath + '/' + file

    file_name = re.compile(r'^(.+)\/([^/]+)\/([^/]+)').sub(r'\3', REIS_path) # group 3 to get file name 
    market_name = re.compile(r'^(.+)\/([^/]+)\/([^/]+)').sub(r'\2', REIS_path) # group 2 to get market name 
    file_save_name = market_name + '_market_data.csv'
    save_path = os.path.join(dirpath, file_save_name)
    #print(save_path)
    if 'trends' in file_name: 
      trends_path_list.append(REIS_path) # to get list of paths to the trends files
    elif 'REIS' in file_name:
      REIS_path_list.append(REIS_path)

  if len(REIS_path_list) > 1: # if you're in a market with more than 1 REIS csv, then there will also be > 1 trends file as well  
    pass
  else:
    try: #to pass the list with nothing in them 
      REIS_df = transform_REIS(REIS_path_list[0])
      trends_df = transform_trends(trends_path_list[0])
      joined_df = join_REIS_trends(REIS_df, trends_df)
      if 'Cap Rate' in joined_df.columns and 'Expenses %' in joined_df.columns:
        exp_and_cap_list.append(joined_df)
        print(market_name)
        #print(joined_df.head())
    except:
      pass
    
    
    

 
exp_and_cap_df = pd.concat(exp_and_cap_list, axis = 1)
exp_and_cap_df = exp_and_cap_df.drop(exp_and_cap_df.columns.difference(['Cap Rate','Expenses %']), axis = 1).groupby(level = 0, axis = 1).mean()
exp_and_cap_df 

In [ ]:
#pd.set_option('display.max_columns', 20)

# to catch if market has more than 1 file with REIS | trends in it 
dir = ''

# mkt_name_REIS_joined.csv = regex parse market path from REIS_path up to last '/' + regex parse market name + '.csv'

for dirpath, dirnames, filenames in os.walk(dir):
  trends_path_list = []
  REIS_path_list = []
  
  trends_df_list = []
  REIS_df_list = []

  #market_name = ''
  for file in filenames:
    
    REIS_path = dirpath + '/' + file

    file_name = re.compile(r'^(.+)\/([^/]+)\/([^/]+)').sub(r'\3', REIS_path) # group 3 to get file name 
    market_name = re.compile(r'^(.+)\/([^/]+)\/([^/]+)').sub(r'\2', REIS_path) # group 2 to get market name 
    file_save_name = market_name + '_market_data.csv'
    save_path = os.path.join(dirpath, file_save_name)
    #print(save_path)
    if 'trends' in file_name: 
      trends_path_list.append(REIS_path) # to get list of paths to the trends files
    elif 'REIS' in file_name:
      REIS_path_list.append(REIS_path)

  if len(REIS_path_list) > 1: # if you're in a market with more than 1 REIS csv, then there will also be > 1 trends file as well 
    # clean trends and REIS df's and append them to lists 
    for REIS_csv_path in REIS_path_list:
      #print(REIS_csv_path)
      REIS_df_list.append(transform_REIS(REIS_csv_path))

    for trends_csv_path in trends_path_list:
      trends_df_list.append(transform_trends(trends_csv_path))

    REIS_sum_df = sum_df_list(REIS_df_list)
    trends_sum_df = sum_df_list(trends_df_list)

    sum_joined_df = join_REIS_trends(REIS_sum_df, trends_sum_df) 
    sum_joined_df['Market Name'] = market_name 

    if 'Cap Rate' not in sum_joined_df.columns and 'Expenses %' not in sum_joined_df.columns:
      sum_joined_df = sum_joined_df.join(exp_and_cap_df, how = 'left')

    sum_joined_df.to_csv(path_or_buf = save_path) 

    print(market_name)
    print(sum_joined_df.head())
  else:
    #don't need to sum up df's. just clean and join 
    try: #to pass the list with nothing in them 
      REIS_df = transform_REIS(REIS_path_list[0])
      trends_df = transform_trends(trends_path_list[0])
    except:
      pass
    joined_df = join_REIS_trends(REIS_df, trends_df)

    if 'Cap Rate' not in joined_df.columns and 'Expenses %' not in joined_df.columns:
      joined_df = joined_df.join(exp_and_cap_df, how = 'left')

    joined_df.to_csv(path_or_buf = save_path)

    print(market_name)
    print(joined_df.head()) 